<h3>Descripción del notebook</h3>
<p>Vamos con la parte de  generar un modelo CNN para entrenar nuestra IA con los datos ya tratados.
Para ello primero cargamos en dataFrames los csv generados con la carga de los datos de las imágenes y sus etiquetas</p>

<p>Cargamos las librerias necesáreas</p>

In [ ]:
import os
import pandas as pd
import pydicom
import numpy as np

from skimage import exposure
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from skimage.transform import resize
from concurrent.futures import ThreadPoolExecutor
from pydicom.pixel_data_handlers import pillow_handler
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import ast

<p>Verifica la versión de TensorFlow y Configuramos para utilizar GPU si está disponible</p>

In [ ]:
print(tf.__version__)

# Configuramos para utilizar GPU si está disponible
if tf.config.experimental.list_physical_devices('GPU'):
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    batch_size = 128
    print("GPU encontrada y configurada para el crecimiento dinámico de la memoria.")
else:
    print("No se encontraron GPUs disponibles.")

<p>Configurar para mostrar todas las columnas y filas
Para ver la matriz total declaramos estas opciones</p>

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

<p>Cargamos los CSV de entrenamiento y prueba</p>

In [ ]:
train_df = pd.read_csv('resultadosXytrain.csv', delimiter=';')
test_df = pd.read_csv('resultadosXytest.csv', delimiter=';')

<p>Convertimos la columna 'Image' de lista a matriz numpy</p>

In [ ]:
train_df['Image'] = train_df['Image'].apply(eval)
print(train_df.shape)
test_df['Image'] = test_df['Image'].apply(eval)

X_train = np.array(train_df['Image'].tolist())
X_test = np.array(test_df['Image'].tolist())

<p>Convertimos las etiquetas a números usando LabelEncoder</p>

In [ ]:
label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
test_df['Label'] = label_encoder.transform(test_df['Label'])

y_train = train_df['Label']
y_test = test_df['Label']

<p>Cambiar el tamaño de entrada del modelo a (128, 128, 3)</p>

In [ ]:
input_shape = (128, 128, 3)

<p>Mejoramos los resultados Agregando capas convolucionales y pooling y dropout</p>

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),

    layers.Dropout(0.5),  # Ajusta el valor de dropout
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),

    layers.Dropout(0.5),  # Ajusta el valor de dropout
    layers.Dense(4, activation='softmax')
])

<p>Compilamos el modelo</p>

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

<p>Configuramos el generador de aumento de datos</p>

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

<p>Aplicar el aumento de datos a las imágenes de entrenamiento</p>

In [ ]:
datagen.fit(X_train)

<p>Entrenamos el modelo con el generador de datos aumentados</p>

In [ ]:
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=100, validation_data=(X_test, y_test))

<p>Evaluar el modelo en el conjunto de prueba</p>

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nPrecisión en el conjunto de prueba: {test_acc}')
print(f'\nPérdida en el conjunto de prueba: {test_loss}')

<p>Visualizamos precisión y pérdida con matplot</p>

In [ ]:
# Definir early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Visualizar precisión y pérdida
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
